In [1]:
import shutil
import subprocess
from os import path, symlink

import gdx2py
from gamspy_base import directory

import xl2times.main as xt
from xl2times.utils import setup_logger

In [2]:
%pip install gdx2py gamspy-base


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
setup_logger(level=0)
input_dir = "../benchmarks/xlsx/DemoS_003/"
output_dir = "../benchmarks/out/DemoS_003-all/"
times_dir = "../TIMES_model/"

model, config = xt.read_xl(
    inputs=[input_dir],
    output_dir=output_dir,
    regions="",
    include_dummy_imports=True,
    no_cache=False,
)

/scratch/htc/skrishna/xl2times/xl2times/transforms.py:1276: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'EOH' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[
Applying transformations from ~TFM_UPD in SYSSETTINGS: 100%|██████████| 2/2 [00:00<00:00, 163.82it/s]


In [4]:
# Run GAMS on the output DD files
def run_gams(times_folder, out_folder):
    # Copy GAMS scaffolding
    scaffolding_folder = path.join("..", "xl2times", "gams_scaffold")
    shutil.copytree(scaffolding_folder, out_folder, dirs_exist_ok=True)
    # Create link to TIMES source
    if not path.exists(path.join(out_folder, "source")):
        symlink(times_folder, path.join(out_folder, "source"), True)

    gams_path = path.join(directory, "gams")
    gams_cmd = (
        gams_path
        + " scenario.run parmfile=gams.opt --run_name=scenario gdx=scenario O=scenario.lst filecase=4"
    )
    # Run GAMS
    res = subprocess.run(
        gams_cmd.split(),
        cwd=out_folder,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        check=True,
    )
    print(res.stdout)
    return

In [5]:
model.attributes.query('attribute == "COM_IE"')

,region,limtype,process,year,attribute,value,source_filename,module_type,module_name,commodity,timeslice,other_indexes,year2,cg,currency
169,REG1,NaN,NaN,2005,COM_IE,0.9,SysSettings,syssettings,SYSSETTINGS,ELC,ANNUAL,NaN,NaN,NaN,MEURO05


In [6]:
tables = xt.to_tables(config, model)
xt.write_dd_files(tables, config, output_dir)

2025-06-03 20:32:29.637 |  SUCCESS : Excel files successfully converted to DD and written to ../benchmarks/out/DemoS_003-all/


In [7]:
run_gams(times_dir, output_dir)

# Inspect parameters in the solution:
with gdx2py.GdxFile(
    path.join(output_dir, "scenario.gdx"),
    gams_dir="/scratch/htc/skrishna/gams/gams49.3.0_linux_x64_64_sfx",
) as g:
    print(g["REG_OBJ"].values())
    print(g["PAR_COMPRDL"].values())
    print(g["EQE_COMPRD"])

--- Job scenario.run Start 06/03/25 20:32:30 49.6.1 55d34574 LEX-LEG x86 64bit/Linux
--- Applying:
    /scratch/htc/skrishna/xl2times/.venv/lib/python3.12/site-packages/gamspy_base/gmsprmun.txt
--- GAMS Parameters defined
    Input /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/scenario.run
    Output /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/scenario.lst
    PageSize 0
    ScrDir /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/225a/
    SysDir /scratch/htc/skrishna/xl2times/.venv/lib/python3.12/site-packages/gamspy_base/
    InputDir1 /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/source/
    InputDir2 /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/model/
    InputDir3 /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/scenarios/
    GDX /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/scenario.gdx
    SysOut 1
    FileCase 4
    --run_name scenario
Licensee: GAMS Demo, for EULA and demo limitatio

In [8]:
# Modify COM_IE and re-run the scenario
com_ie = model.attributes.query('attribute == "COM_IE"')
model.attributes.loc[com_ie.index, "value"] = 0.95
print(com_ie["value"])

tables = xt.to_tables(config, model)
xt.write_dd_files(tables, config, output_dir)

run_gams(times_dir, output_dir)

# Inspect parameters in the solution:
with gdx2py.GdxFile(
    path.join(output_dir, "scenario.gdx"),
    gams_dir="/scratch/htc/skrishna/gams/gams49.3.0_linux_x64_64_sfx",
) as g:
    print(g["REG_OBJ"].values())
    print(g["PAR_COMPRDL"].values())
    print(g["EQE_COMPRD"])

169    0.9
Name: value, dtype: object
2025-06-03 20:32:34.684 |  SUCCESS : Excel files successfully converted to DD and written to ../benchmarks/out/DemoS_003-all/


--- Job scenario.run Start 06/03/25 20:32:34 49.6.1 55d34574 LEX-LEG x86 64bit/Linux
--- Applying:
    /scratch/htc/skrishna/xl2times/.venv/lib/python3.12/site-packages/gamspy_base/gmsprmun.txt
--- GAMS Parameters defined
    Input /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/scenario.run
    Output /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/scenario.lst
    PageSize 0
    ScrDir /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/225a/
    SysDir /scratch/htc/skrishna/xl2times/.venv/lib/python3.12/site-packages/gamspy_base/
    InputDir1 /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/source/
    InputDir2 /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/model/
    InputDir3 /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/scenarios/
    GDX /scratch/htc/skrishna/xl2times/benchmarks/out/DemoS_003-all/scenario.gdx
    SysOut 1
    FileCase 4
    --run_name scenario
Licensee: GAMS Demo, for EULA and demo limitatio